In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

In [35]:
# set current time and date
def options_arbitrage(ticker, expiration_date, print_tables=False, to_latex=False):
    # set time
    now = datetime.now()
    
    # select the stock
    stock = yf.Ticker(ticker)

    # get latest market price
    price = stock.info['currentPrice']

    # get the option chain for the expiration date
    opt = stock.option_chain(expiration_date)
    
    # get the calls and puts
    calls = opt.calls
    puts = opt.puts

    # change time to 
    calls['lastTradeDate'] = calls['lastTradeDate'].dt.tz_localize(None).dt.date
    puts['lastTradeDate'] = puts['lastTradeDate'].dt.tz_localize(None).dt.date

    calls = calls[['contractSymbol','lastTradeDate','strike', 'lastPrice', 'bid', 'ask', 'volume']].rename(columns={'strike': 'strike', 'lastPrice': 'call_lastPrice', 'bid': 'call_bid', 'ask': 'call_ask', 'volume': 'call_volume', 'lastTradeDate': 'call_lastTradeDate', 'contractSymbol': 'call_contractSymbol'})
    puts = puts[['contractSymbol','lastTradeDate','strike', 'lastPrice', 'bid', 'ask', 'volume']].rename(columns={'strike': 'strike', 'lastPrice': 'put_lastPrice', 'bid': 'put_bid', 'ask': 'put_ask', 'volume': 'put_volume', 'lastTradeDate': 'put_lastTradeDate', 'contractSymbol': 'put_contractSymbol'})

    merge = pd.merge(left=calls, right=puts, left_on='strike', right_on='strike', how='inner')

    # check length of the merge has at least 2 rows
    if len(merge) < 2:
        return ValueError('Not enough data to calculate put-call parity for 2 strike prices')

    strike1 = merge[merge['strike'] < price].iloc[-1]['strike']
    strike2 = merge[merge['strike'] > price].iloc[0]['strike']


    # filter merge, calls and puts to only include the relevant strike prices
    relevant_merge = merge[(merge['strike'] == strike1) | (merge['strike'] == strike2)].reset_index(drop=True)
    relevant_calls = calls[(calls['strike'] == strike1) | (calls['strike'] == strike2)].reset_index(drop=True)
    relevant_puts = puts[(puts['strike'] == strike1) | (puts['strike'] == strike2)].reset_index(drop=True)
    
    # add stock price
    relevant_merge['stock_price'] = price

    # calculate the put-call parity
    relevant_merge['parity'] = relevant_merge['call_bid'] + relevant_merge['strike'] - (relevant_merge['put_bid'] + relevant_merge['stock_price'])

    # add dowload time
    relevant_merge['dowload_time'] = now 

    # reorder the columns
    merge_latex = relevant_merge[['call_bid', 'strike', 'put_bid', 'stock_price', 'parity', 'dowload_time']]
    merge_latex = merge_latex.rename(columns={'call_bid': '$C_0$', 'strike': '$X$', 'put_bid': '$P_0$', 'stock_price': '$S_0$', 'parity': 'Deviation', 'dowload_time': 'Download Time'})

    if print_tables:
        print(f'Call Options for {ticker} with expiration date {expiration_date}')
        display(relevant_calls)
        print(f'Put Options for {ticker} with expiration date {expiration_date}')
        display(relevant_puts)
        print(f'Put-Call Parity for {ticker} with expiration date {expiration_date}')
        display(merge_latex)
    
    if to_latex:        

        latex_call = relevant_calls.to_latex(index=False, float_format='%.2f', caption=f'Call Options for {ticker} with expiration date {expiration_date}')
        latex_put = relevant_puts.to_latex(index=False, float_format='%.2f', caption=f'Put Options for {ticker} with expiration date {expiration_date}')
        latex_merge = merge_latex.to_latex(index=False, float_format='%.2f', caption=f'Put-Call Parity for {ticker} with expiration date {expiration_date}')

        # replace toprule, midrule, bottomrule with hline
        latex_call = latex_call.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline')
        latex_put = latex_put.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline')
        latex_merge = latex_merge.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline')

        latex_call = latex_call.replace('call_','').replace('begin{table}', 'begin{table}[H]\n\centering').replace('+00:00','')
        latex_put = latex_put.replace('put_','').replace('begin{table}', 'begin{table}[H]\n\centering').replace('+00:00','')
        latex_merge = latex_merge.replace('begin{table}', 'begin{table}[H]\n\centering')

        print(latex_call)
        print(latex_put)
        print(latex_merge)

In [36]:
options_arbitrage(ticker='LMT', expiration_date='2024-03-08', print_tables=False, to_latex=True)

\begin{table}[H]
\centering
\caption{Call Options for LMT with expiration date 2024-03-08}
\begin{tabular}{llrrrrr}
\hline
contractSymbol & lastTradeDate & strike & lastPrice & bid & ask & volume \\
\hline
LMT240308C00425000 & 2024-02-12 & 425.00 & 9.30 & 8.10 & 8.60 & 4.00 \\
LMT240308C00430000 & 2024-02-12 & 430.00 & 5.90 & 5.20 & 5.80 & 46.00 \\
\hline
\end{tabular}
\end{table}

\begin{table}[H]
\centering
\caption{Put Options for LMT with expiration date 2024-03-08}
\begin{tabular}{llrrrrr}
\hline
contractSymbol & lastTradeDate & strike & lastPrice & bid & ask & volume \\
\hline
LMT240308P00425000 & 2024-02-12 & 425.00 & 5.60 & 5.30 & 6.10 & 5.00 \\
LMT240308P00430000 & 2024-02-09 & 430.00 & 9.90 & 8.00 & 8.50 & 2.00 \\
\hline
\end{tabular}
\end{table}

\begin{table}[H]
\centering
\caption{Put-Call Parity for LMT with expiration date 2024-03-08}
\begin{tabular}{rrrrrl}
\hline
$C_0$ & $X$ & $P_0$ & $S_0$ & Deviation & Download Time \\
\hline
8.10 & 425.00 & 5.30 & 428.07 & -0.27 & 2